In [ ]:
# https://ai.google.dev/edge/litert/conversion/tensorflow/quantization/post_training_quantization#integer_only
# https://github.com/google-coral/tutorials/blob/master/retrain_classification_ptq_tf2.ipynb

#GOOGLE COLAB CONVERSION CODE
import tensorflow as tf
import numpy as np
import pandas as pd
from google.colab import files
import os

MODEL_PATH = '/content/ferplus_model_pd_best.h5'
CSV_PATH = '/content/fer2013.csv'

# A custom SeparableConv2D class to handle old arguments for Keras 3 compatibility
class CustomSeparableConv2D(tf.keras.layers.SeparableConv2D):
    def __init__(self, *args, kernel_initializer=None, kernel_regularizer=None,
                 kernel_constraint=None, groups=1, **kwargs):
        # Remap old kernel_initializer to depthwise_initializer and pointwise_initializer
        if kernel_initializer and 'depthwise_initializer' not in kwargs:
            kwargs['depthwise_initializer'] = kernel_initializer
        if kernel_initializer and 'pointwise_initializer' not in kwargs:
            kwargs['pointwise_initializer'] = kernel_initializer

        # Remap old kernel_regularizer to depthwise_regularizer and pointwise_regularizer
        if kernel_regularizer and 'depthwise_regularizer' not in kwargs:
            # Ensure the regularizer is correctly instantiated for Keras 3
            if isinstance(kernel_regularizer, dict) and kernel_regularizer.get('class_name') == 'L2':
                # Keras 3's L2 regularizer expects 'l2_regularization_factor'
                l2_value = kernel_regularizer.get('config', {}).get('l2')
                if l2_value is not None:
                    kwargs['depthwise_regularizer'] = tf.keras.regularizers.L2(l2=l2_value)
                    kwargs['pointwise_regularizer'] = tf.keras.regularizers.L2(l2=l2_value)
                else:
                    kwargs['depthwise_regularizer'] = tf.keras.regularizers.get(kernel_regularizer)
                    kwargs['pointwise_regularizer'] = tf.keras.regularizers.get(kernel_regularizer)
            else:
                kwargs['depthwise_regularizer'] = tf.keras.regularizers.get(kernel_regularizer)
                kwargs['pointwise_regularizer'] = tf.keras.regularizers.get(kernel_regularizer)


        # Remap old kernel_constraint to depthwise_constraint and pointwise_constraint
        if kernel_constraint and 'depthwise_constraint' not in kwargs:
            kwargs['depthwise_constraint'] = kernel_constraint
        if kernel_constraint and 'pointwise_constraint' not in kwargs:
            kwargs['pointwise_constraint'] = kernel_constraint

        # 'groups' argument is usually 1 for SeparableConv2D, safely ignore if present
        super().__init__(*args, **kwargs)

# A custom SpatialDropout2D class to handle old arguments for Keras 3 compatibility
class CustomSpatialDropout2D(tf.keras.layers.SpatialDropout2D):
    def __init__(self, *args, **kwargs):
        # Remove 'trainable' and 'noise_shape' if present, as they are not constructor arguments for SpatialDropout2D in Keras 3
        kwargs.pop('trainable', None)
        kwargs.pop('noise_shape', None)
        super().__init__(*args, **kwargs)

# Load the Keras model from the .h5 file, setting compile=False to avoid issues with old optimizer/loss configs
with tf.keras.utils.custom_object_scope({'SeparableConv2D': CustomSeparableConv2D, 'SpatialDropout2D': CustomSpatialDropout2D}):
    saved_model_dir = tf.keras.models.load_model(MODEL_PATH, compile=False)

print("Model loaded successfully from:", MODEL_PATH)
print("\nModel input shape:", saved_model_dir.input_shape)
print("Model output shape:", saved_model_dir.output_shape)


# Initialize the TFLite converter
converter = tf.lite.TFLiteConverter.from_keras_model(saved_model_dir)

# Apply optimizations for quantization and set up INT8 quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8

# Provide the representative dataset for calibration
# Load samples from CSV
print(f"\nLoading samples from CSV: {CSV_PATH}")
# Load FER2013 CSV
df = pd.read_csv(CSV_PATH)

# Use training data for calibration
train_df = df[df['Usage'] == 'Training']

# Define num_samples
num_samples = 100 # You can adjust this number based on your needs

# Sample random rows
sample_df = train_df.sample(n=min(num_samples, len(train_df)), random_state=42)

calibration_images = [] # final calibration images for representative, initialized as a list

# Convert pixel strings to images
for pixel_string in sample_df['pixels']:
    # Convert space-separated string to numpy array
    pixel_array = np.fromstring(pixel_string, dtype=np.uint8, sep=' ')
    # Reshape to 48x48 and normalize
    image = pixel_array.reshape(48, 48, 1).astype(np.float32) / 255.0
    calibration_images.append(image)

# Convert list to numpy array after appending all images
calibration_images = np.array(calibration_images)

# Generator function
def representative_dataset_gen():
    for i in range(len(calibration_images)):
        sample = calibration_images[i:i+1].astype(np.float32)
        yield [sample]

# Use in converter
converter.representative_dataset = representative_dataset_gen

# Convert the model
print("\nQuantization settings:")
print(f"  - Optimization: {converter.optimizations}")
print(f"  - Target ops: INT8")
print(f"  - Input type: INT8")
print(f"  - Output type: INT8")
print(f"  - Calibration samples: {len(calibration_images)}") # Corrected variable name

# Convert the model
print("\nConverting model... (this may take a minute)")
try:
    tflite_quant_model = converter.convert()
    print("✓ Conversion successful!")
except Exception as e:
    print(f"✗ Conversion failed: {e}")
    raise

# Save the TFLite model to a file on the Colab filesystem
tflite_model_filename = '/content/converted_model_int8.tflite'
with open(tflite_model_filename, 'wb') as f:
    f.write(tflite_quant_model)

print("Model successfully converted and saved to:", tflite_model_filename)

# Download the model to your local machine
files.download(tflite_model_filename)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_separable_conv.py:104: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
/usr/local/lib/python3.12/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model loaded successfully from: /content/ferplus_model_pd_best.h5

Model input shape: (None, 48, 48, 1)
Model output shape: (None, 8)

Loading samples from CSV: /content/fer2013.csv

Quantization settings:
  - Optimization: [<Optimize.DEFAULT: 'DEFAULT'>]
  - Target ops: INT8
  - Input type: INT8
  - Output type: INT8
  - Calibration samples: 100

Converting model... (this may take a minute)
Saved artifact at '/tmp/tmp21nlb32x'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 48, 48, 1), dtype=tf.float32, name='separable_conv2d_20_input')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  136107966232720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136107966231568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136107960367632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136107960368016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136107960367056: TensorSpec(

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


✓ Conversion successful!
Model successfully converted and saved to: /content/converted_model_int8.tflite


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>